# Importing Libraries and Loading Data:


This section imports the necessary libraries, downloads NLTK data, initializes a WordNetLemmatizer, loads and preprocesses the dataset, tokenizes sentences, and calculates the TF-IDF matrix for the text data.

In [115]:
# Import necessary libraries
import random
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from ipywidgets import widgets, Layout
from IPython.display import display, HTML

In [116]:
# Load NLTK data
nltk.download('popular', quiet=True)
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\B.Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\B.Traders\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [117]:
# Initialize WordNetLemmatizer
lemmer = WordNetLemmatizer()

# Load and preprocess the dataset
with open('tourism data.txt', 'r', errors='ignore') as f:
    raw_data = f.read().lower()

# Tokenize sentences and preprocess tokens
sentence_tokens = nltk.sent_tokenize(raw_data)

# Lemmatize tokens
def LemmedTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# Normalize and lemmatize tokens
def NormalizedLemmedTokens(text):
    return LemmedTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Calculate TF-IDF matrix
TfidfVec = TfidfVectorizer(tokenizer=NormalizedLemmedTokens, stop_words='english')
tfidf = TfidfVec.fit_transform(sentence_tokens)


# Defining Response Generation Function:

This section defines the response function, which generates a response based on user input. It calculates cosine similarity, handles "tell me about" queries, and matches keywords to sentences in the dataset.

In [118]:
def response(user_response):
    user_response = user_response.lower()
    sentence_tokens.append(user_response)
    tfidf = TfidfVec.fit_transform(sentence_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if req_tfidf == 0:
        return "I am sorry! I don't understand you"
    else:
        if 'tell me about' in user_response:
            query = user_response.replace('tell me about', '').strip()
            for sentence in sentence_tokens:
                if query in sentence:
                    return sentence
        else:
            matches = [sentence for sentence in sentence_tokens if any(keyword in sentence for keyword in user_response.split())]
            if matches:
                return random.choice(matches)
            else:
                return "I am sorry! I don't have information about that destination."


# Defining Greeting and Farewell Functions:

This section defines functions for greeting the user and bidding farewell.

In [119]:
def greet_user():
    responses = ["Hello! I'm your TravelBot. How can I help you today?",
                 "Hi there! Welcome to TravelBot. What would you like to know?",
                 "Greetings! I'm TravelBot. Feel free to ask me about travel destinations."]
    return random.choice(responses)

def farewell():
    return "Goodbye! Have a great day and safe travels!"


# Creating User Interface:

This section creates the user interface elements using the ipywidgets library for displaying output, taking user input, and sending messages.

In [120]:
html_output = widgets.Output()
html_output.layout.width = '600px'
html_output.layout.border = '1px solid #ccc'
html_output.layout.padding = '10px'

user_input = widgets.Text(placeholder='Type your message...', layout=Layout(width='80%'))
send_button = widgets.Button(description='Send', button_style='success', layout=Layout(width='20%'))
user_input_box = widgets.HBox([user_input, send_button])


# Defining Message Sending and Receiving Function:

This section defines the send_message function, which processes user input, generates responses, and displays the conversation in the HTML output area.

In [121]:
def send_message(button):
    user_message = user_input.value
    user_input.value = ''  # Reset the input text
    with html_output:
        display(HTML(f'<p><strong>You:</strong> {user_message}</p>'))
        
        if user_message.lower() == 'bye':
            print("TravelBot:", farewell())
            user_input.disabled = True
            send_button.disabled = True
        elif any(greeting_word in user_message.lower() for greeting_word in ["hello", "hi", "greetings", "sup", "what's up","hey"]):
            greeting_response = greet_user()
            display(HTML(f'<p><strong>TravelBot:</strong> {greeting_response}</p>'))
        else:
            bot_response = response(user_message)
            display(HTML(f'<p><strong>TravelBot:</strong> {bot_response}</p>'))

send_button.on_click(send_message)


# Displaying Interface:


This section displays the initial greeting message and the user interface 

In [122]:
init_message = greet_user()
with html_output:
    display(HTML(f'<p><strong>TravelBot:</strong> {init_message}</p>'))

display(html_output)
display(user_input_box)


Output(layout=Layout(border='1px solid #ccc', padding='10px', width='600px'))